In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import folium
from folium import plugins
import fancyimpute

Using TensorFlow backend.


In [2]:
years = [2009,2010,2011,2012,2013,2014,2015]
path ='/public/npatil4/dataMining/data/daily/combined_wide/'
files = [path+'combined_wide_'+str(year)+'.csv' for year in years]

In [3]:
data = [pd.read_csv(file_name,sep='|') for file_name in files]

In [4]:
[df.shape for df in data]

[(148428, 18),
 (160416, 18),
 (157975, 18),
 (168291, 18),
 (184197, 18),
 (196085, 18),
 (206363, 18)]

In [5]:
data= pd.concat(data, ignore_index=True)

In [8]:
data['DateLocal'] = pd.to_datetime(data.DateLocal)
data['year'] = data.DateLocal.dt.year
data['weekNo'] = data.DateLocal.dt.week
data['Weekday'] = data.DateLocal.dt.weekday

In [9]:
data.head(2).transpose()

,0,1
DateLocal,2009-01-01 00:00:00,2009-01-01 00:00:00
CityName,Aberdeen,Adjuntas
CountyName,Brown,Adjuntas
StateName,South Dakota,Puerto Rico
Latitude,45.4625,18.17538
Longitude,-98.48611,-66.72599
Barometricpressure,NaN,NaN
Carbonmonoxide,NaN,NaN
DewPoint,NaN,NaN
NitrogenDioxide,NaN,NaN


In [10]:
data_missing = data.apply(pd.isnull)

In [11]:
100 * data_missing.ix[:,6:-4].apply(sum)/data_missing.shape[0]

Barometricpressure    81.756326
Carbonmonoxide        76.663324
DewPoint              98.023335
NitrogenDioxide       70.833923
OutdoorTemperature    66.338505
Ozone                 54.405752
PM                     0.000000
RelativeHumidity      78.592762
Sulfurdioxide         72.999251
WindSpeed             71.541185
dtype: float64

In [12]:
data_missing_count = data_missing.ix[:,6:-4].apply(sum,1).groupby([data.CountyName,
                                                                   data.CityName]
                                                                 ).mean()
data_missing_count = pd.DataFrame(data_missing_count[data_missing_count>=8]).reset_index()

In [13]:
data_pol = data.ix[:,:-4].groupby([
       'CityName', 'CountyName',
        'Latitude','Longitude'],
        as_index=False).mean()

In [14]:
data_pol = data.ix[:,:-4].groupby([
       'DateLocal',
       'CityName', 'CountyName',
       'Latitude','Longitude'],
        as_index=False).mean()


In [15]:
columns = [
 'Carbonmonoxide',
 'DewPoint',
 'NitrogenDioxide',
 'OutdoorTemperature',
 'Ozone',
 'PM',
 'RelativeHumidity',
 'Sulfurdioxide'
]

In [16]:
X_incomplete = data_pol.ix[:,columns]
X_incomplete.head()

,Carbonmonoxide,DewPoint,NitrogenDioxide,OutdoorTemperature,Ozone,PM,RelativeHumidity,Sulfurdioxide
0,NaN,NaN,NaN,NaN,NaN,5.20,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,3.90,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,7.90,NaN,2.576708
3,NaN,NaN,NaN,NaN,NaN,7.90,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,4.95,NaN,NaN


In [17]:
X_incomplete.shape

(1221755, 8)

In [18]:
X_incomplete.to_csv(
    '../data/daily/combined_wide/combined_wide_all_missing.txt',
    sep='|')
